# Is the House Price in Metro Atlanta Related to the Convinence 

## Introduction

Many people might think that the house price of an area is positive correlated to living requirements but actually it is not always correct. In this project, I explored the house price and venues in cities in Metro Atlanta and visualized the result to show the correlation.

### Data

**The list of cities in Metro Atlanta to locate the cities and to search the venues around cities.**

I scraped the list from this [website](https://support.crunchbase.com/hc/en-us/articles/360009896314-What-cities-are-in-the-Greater-Atlanta-Area-region-).

**The restaurant information, including names, categories, latitude, and longitude, to summerize the venues in each city and check the distribution of venues.**

The data is generated for [forsquare API](https://foursquare.com/).

**House price in each area of Atlanta.**

I obained the data from [Zillow](https://www.zillow.com/).

### Scraping City List and Visualization

In [1]:
from bs4 import BeautifulSoup     
import requests 

In [2]:
url = 'https://support.crunchbase.com/hc/en-us/articles/360009896314-What-cities-are-in-the-Greater-Atlanta-Area-region-'
web = requests.get(url).text

In [3]:
soup = BeautifulSoup(web)
loc2 = soup.find('section', class_ = 'article-info')
contents = loc2.find_all('li')

In [4]:
city = []
for row in contents:
    loc = row.string
    city.append(loc)

In [5]:
city.append('Doraville')
city.append('Sandy Springs')
city_all = [city[i] +', GA' for i in range(len(city))]

In [7]:
from geopy.geocoders import Nominatim
import pandas as pd
lat = []
lng = []
for i in range(len(city_all)):
    geolocator = Nominatim(user_agent='atl_explore')
    location = geolocator.geocode(city_all[i])
    lat.append(location.latitude)
    lng.append(location.longitude)

In [9]:
df = pd.DataFrame({'City':city, 'Latitude': lat, 'Longitude': lng})

In [10]:
import folium
geolocator = Nominatim(user_agent='atl_explore')
location = geolocator.geocode('Atlanta, GA')
lat_atl = location.latitude
lng_atl = location.longitude
# map 
map_city = folium.Map(location=[lat_atl, lng_atl], zoom_start=8)
for lat, lng in zip(df['Latitude'], df['Longitude']):
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_city)      
map_city

In [11]:
from geopy.distance import geodesic
dis = []
for i in range(df.shape[0]):
    dis.append(geodesic((lat_atl,lng_atl), (df['Latitude'][i], df['Longitude'][i])).miles)
df['Distance'] = dis

In [12]:
def color_producer(dis):
    if dis < 25:
        return 'red'
    else:
        return 'blue'
map_city = folium.Map(location=[lat_atl, lng_atl], zoom_start=8)

for lat, lng, dis in zip(df['Latitude'], df['Longitude'], df['Distance']):
    folium.CircleMarker(
        [lat, lng],
        radius=8,
        color=color_producer(dis),
        fill=True,
        fill_color=color_producer(dis),
        fill_opacity=0.7,
        parse_html=False).add_to(map_city)  
    
map_city

In [13]:
df.rename(columns={'Unnamed: 0':'City'}, inplace = True)

In [14]:
df_nearby = df[df['Distance'] < 25]

### Obtain Venues in Selected Cities

In [15]:
import requests
from pandas.io.json import json_normalize 

In [16]:
# account information
CLIENT_ID = 'KGL3AWGZJOTUDQJ1OH03XE3XJD1YAYJVS55Y5EW14IM4DCDO' 
CLIENT_SECRET = 'X3NKM45152MTWIGHWNPJUYZKUVEKBWOM3VJCEZ0VX4RD4DAA' 
VERSION = '20180605' 

In [17]:
def get_venues(city_name):
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, CLIENT_SECRET, VERSION, df['Latitude'][city_name], df['Longitude'][city_name], 1000, 100)
    result = requests.get(url).json()
    venue = result['response']['groups'][0]['items'] 
    try:
        venue_city = json_normalize(venue).loc[:,['venue.name', 'venue.categories', 'venue.location.city', 'venue.location.lat', 'venue.location.lng']]
        venue_city.columns = ['Name', 'Categories', 'City', 'Latitude', 'Longitude']
        for i in range(venue_city.shape[0]):
            venue_city['Categories'][i]= venue_city['Categories'][i][0]['name']
        return(venue_city)
    except:
        pass   

In [18]:
city_new = df_nearby.index
venue_all = pd.DataFrame(columns = ['Name', 'Categories', 'City', 'Latitude', 'Longitude'])
for c in city_new:
    venue = get_venues(c)
    venue_all = venue_all.append(venue, ignore_index = True)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  import sys
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [19]:
venue_all.dropna(inplace = True)
venue_all.drop_duplicates(keep = 'first', inplace = True, ignore_index = True)

### EDA for Venues

In [20]:
venue_all.groupby('City').count()

,Name,Categories,Latitude,Longitude
City,,,,
Alpharetta,67,67,67,67
Atlanta,210,210,210,210
Atlanta-Decatur,1,1,1,1
Austell,19,19,19,19
Avondale Estates,25,25,25,25
Brentwood,2,2,2,2
Chamblee,58,58,58,58
Chamblee-Doraville,1,1,1,1
Clarkston,11,11,11,11


In [21]:
top25 = venue_all.groupby('City')['Name'].count().sort_values(ascending = False).index[1:26].tolist()
df_nearby.set_index('City', inplace = True)
df_25 = df_nearby.loc[top25,:]
df_25['Venues'] = venue_all.groupby('City')['Name'].count().sort_values(ascending = False)[1:26]

In [153]:
def color_producer(number):
    if number >= 100:
        return 'red'
    elif number >= 50:
        return 'green'
    else:
        return 'blue'
map_25 = folium.Map(location = [lat_atl, lng_atl], zoom_start = 10)
for lat, lng, venue in zip(df_25['Latitude'], df_25['Longitude'], df_25['Venues']):
    folium.CircleMarker(
    [lat, lng],
    radius = 20,
    color = color_producer(venue),
    fill_color = color_producer(venue),
    fill_opacity = 0.5 ).add_to(map_25)
map_25

In [150]:
index_south = (df_25['Latitude'] < lat_atl)
df_north = df_25[~index_south]
df_north.shape
# Drop south cities from restaurant list
keys = np.where(index_south == True)
south_city = df_25.index[keys]
for i in south_city:
    venue_north = venue_all.drop(venue_all[venue_all['City'] == i].index, axis = 0)
venue_north.reset_index(drop = True, inplace = True)
city_list = df_25.index[~index_south]

In [151]:
dummy = pd.get_dummies(venue_north['Categories'])
dummy['City'] = venue_north['City']
group_sum = dummy.groupby('City').sum()
group_sum=group_sum.reset_index()
group_sum

,City,ATM,Accessories Store,African Restaurant,Airport Terminal,American Restaurant,Antique Shop,Arepa Restaurant,Art Gallery,Arts & Crafts Store,...,Vietnamese Restaurant,Water Park,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio
0,Alpharetta,0,0,0,0,4,0,0,0,0,...,1,0,0,0,0,0,1,0,2,0
1,Atlanta,0,0,0,1,7,1,0,0,1,...,0,0,1,1,0,1,0,1,1,3
2,Atlanta-Decatur,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Austell,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Avondale Estates,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0
5,Brentwood,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Chamblee,0,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Chamblee-Doraville,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Clarkston,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Clayton County,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [152]:
num_top_venues = 5

for c in city_list:
    print("----"+c+"----")
    temp = group_sum[group_sum['City'] == c].T.reset_index()
    temp.columns = ['venue','count']
    temp = temp.iloc[1:]
    print(temp.sort_values('count', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Decatur----
         venue count
0  Pizza Place     8
1          Spa     5
2  Coffee Shop     5
3          Pub     4
4    Gastropub     4


----Alpharetta----
                     venue count
0           Clothing Store     8
1      American Restaurant     4
2              Coffee Shop     3
3  New American Restaurant     3
4   Furniture / Home Store     3


----Roswell----
                       venue count
0                Pizza Place     4
1         Italian Restaurant     3
2        American Restaurant     3
3         Mexican Restaurant     3
4  Cajun / Creole Restaurant     2


----Chamblee----
                    venue count
0      Chinese Restaurant     4
1  Furniture / Home Store     4
2          Sandwich Place     3
3                Pharmacy     2
4  Thrift / Vintage Store     2


----Stone Mountain----
               venue count
0        Gas Station     3
1              Hotel     3
2           Mountain     3
3              Trail     3
4  Convenience Store     2


----Snellvi

Highlights:

1. Top 5 venues in **Roswell** are all restaurants!!!

2. **Sandy Springs** is a vegetarian-friendly place (3 vegan restaurants).

3. **Decatur** has many pizza places and coffee shops.

4. **Chamblee** is the hub of chinese restaurants. It has 4 Chinese restaurants and 2 dim sum restaurants.

5. **Alpharetta** has 8 American restaurants.

### EDA for House Price

In [23]:
hp = pd.read_csv('/Users/chen-tze/Desktop/Neighborhood_Zhvi_AllHomes.csv')
ga = hp[hp['State']=='GA']
ga.reset_index(drop = True, inplace = True)

In [24]:
# least yearly price
ga['avg_yr_price'] = ga.iloc[:, -12:].mean(axis=1)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [25]:
from opencage.geocoder import OpenCageGeocode
key = '8978fb3e3a324791aae01fa70e9bd524'
geocoder = OpenCageGeocode(key)

In [26]:
ga['Location'] = ga['RegionName'] + ', ' + ga['City']

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [35]:
import numpy as np
lat = []
lng = []
for i in ga['Location']:
    results = geocoder.geocode(i)    
    latitude = results[0]['geometry']['lat']
    longitude = results[0]['geometry']['lng']
    lat.append(latitude)
    lng.append(longitude)
    print(i)

Midtown, Atlanta
Downtown, Atlanta
Belair, Augusta
Old Fourth Ward, Atlanta
Jamestown, Augusta
Georgetown, Savannah
North Springs, Sandy Springs
Southside, Augusta
North Buckhead, Atlanta
Morningside - Lenox Park, Atlanta
Richmond Hill, Augusta
Windsor Spring, Augusta
Pine Hills, Atlanta
Grant Park, Atlanta
Home Park, Atlanta
Wilshire Estates-Savannah Mall, Savannah
Wheeless Road, Augusta
Virginia Highland, Atlanta
West Augusta, Augusta
Kirkwood, Atlanta
Meadowbrook, Augusta
Lake Aumond, Augusta
Windsor Forest, Savannah
Pepperidge, Augusta
Richmond Factory, Augusta
Atlanta University Center, Atlanta
Georgia Tech, Atlanta
Ashford Park, Brookhaven
Highland Park, Augusta
Old Savannah, Augusta
Glenrose Heights, Atlanta
Barton Chapel, Augusta
West End, Atlanta
Lindbergh, Atlanta
Montclair, Augusta
National Hills, Augusta
Ormewood Park, Atlanta
East Atlanta, Atlanta
Greenbriar, Atlanta
Sylvan Hills, Atlanta
Summerville, Augusta
North Leg, Augusta
Oakhurst, Decatur
Edgewood, Atlanta
Lindridge

Edgemere, Savannah
Atkins Park, Atlanta
Aberdeen Forest, Sandy Springs
Deerwood, Atlanta
Elmco Estates, Atlanta
Bennett Woods, Smyrna
Kensington Park, Savannah
Addison Heights, Marietta
Rosedale Heights, Atlanta
Baker Hills, Atlanta
Sunset Park, Savannah
Woodland Hills, Atlanta
South Side, Valdosta
Skyland, Brookhaven
Clairemont Ave, Decatur
Ridgewood Heights, Atlanta
Wynnton Grove, Columbus
Oak Creek Estates, Marietta
Kayton-Frazier Area, Savannah
Hillsdale, Brookhaven
MAK Historic District, Decatur
Winter Gardens-Brightwood, Savannah
High Point, Atlanta
Fernwood-Parkwood, Savannah
Florida Heights, Atlanta
Meadowbrook Forest, Atlanta
Boulevard Heights, Atlanta
Hampton Hall, Brookhaven
Old Decatur, Decatur
Arlington Estates, Atlanta
Briar Glen, Atlanta
King Valley at Vinnings, Smyrna
Whittier Mill Village, Atlanta
Blair Villa - Poole Creek, Atlanta
Wyngate, Atlanta
Lenox Place, Decatur
Glenridge, Sandy Springs
Cherokee Trail, Marietta
Wisteria Gardens, Atlanta
Whitewater Creek, Atlanta

In [40]:
ga.loc[:,'Latitude'] = lat
ga.loc[:,'Longitude'] = lng
ga.loc[:,'std_price'] = (ga.loc[:,'avg_yr_price'] - ga['avg_yr_price'].min())/(ga['avg_yr_price'].max()-ga['avg_yr_price'].min())

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [ ]:
ga['avg_yr_price'] = ga.iloc[:, -12:].mean(axis=1)

In [69]:
def color_producer(number):
    if number >= 0.5:
        return 'red'
    elif number >= 0.3:
        return 'green'
    else:
        return 'blue'
p_map = folium.Map(location = [lat_atl, lng_atl], zoom_start = 10)
for lng, lat, p in zip(ga['Longitude'], ga['Latitude'], ga['std_price']):
    folium.CircleMarker(location = [lat, lng],
                 radius = p*20,
                 color = color_producer(p),
                 fill = True,
                 fill_opacity = 0.3, 
                 fill_color = color_producer(p)).add_to(p_map)
    #popup=ga.iloc[i]['RegionName'],
p_map



The map showed that the house price in the area near north west of Atlant is higher.

### Comparison of Venues and House Price

In [70]:
n_5 = ['Roswell', 'Alpharetta', 'Duluth', 'Chamblee', 'Sandy Springs']
df_5 = df_25.loc[n_5, ['Latitude', 'Longitude']]
df_decatur = df_25.loc['Decatur',['Latitude', 'Longitude']]

In [106]:
ga_p = ga[ga['std_price']>=0.5]
ga_p.sort_values('Longitude', inplace = True)
ga_atl = ga_p.iloc[0:16, -4:-1]
ga_atl.reset_index(drop = True, inplace = True)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [158]:
m  = folium.Map(location = [lat_atl, lng_atl], zoom_start = 10)
folium.PolyLine([[ga_atl.iloc[5,1],ga_atl.iloc[5,2]],
                [ga_atl.iloc[8,1],ga_atl.iloc[15,2]],
                [ga_atl.iloc[11,1],ga_atl.iloc[11,2]],
                [ga_atl.iloc[14,1],ga_atl.iloc[6,2]],
                [ga_atl.iloc[5,1],ga_atl.iloc[5,2]]],
                color = 'purple',
                fill_color = 'purple',
                fill_opacity = 0.5).add_to(m)
folium.PolyLine([[df_5.iloc[0,0],df_5.iloc[0,1]],
                [df_5.iloc[1,0],df_5.iloc[1,1]],
                [df_5.iloc[2,0],df_5.iloc[2,1]],
                [df_5.iloc[3,0],df_5.iloc[3,1]],
                [df_5.iloc[4,0],df_5.iloc[4,1]],
                [df_5.iloc[0,0],df_5.iloc[0,1]]],
                color = 'yellow',
                fill_color = 'yellow',
                fill_opacity = 0.5).add_to(m)
folium.CircleMarker([df_decatur['Latitude'], df_decatur['Longitude']],
                   color = 'yellow',
                   radius = 30,
                   fill_color = 'yellow',
                   fill_opacity = 0.5).add_to(m)
folium.Marker([df_decatur['Latitude'], df_decatur['Longitude']],
              popup = 'Restaruants & Entertainment',
              icon=folium.Icon(color='red', icon='info-sign'),).add_to(m)
folium.Marker([df_5.iloc[:,0].mean(), df_5.iloc[:,1].mean()],
              popup = 'Restaruants & Entertainment',
              icon=folium.Icon(color='blue', icon='info-sign'),).add_to(m)
folium.Marker([(ga_atl.iloc[5,1]+ga_atl.iloc[11,1])/2, ga_atl.iloc[5:15,2].mean()],
              popup = 'Area with High House Price',
              icon=folium.Icon(color='green', icon='info-sign'),).add_to(m)
m

### Conclusion

Based on the maps, we know that the house price and the venues in the north Atlanta is higher than the south. However, the area with the most venues and the area with higher house price are not the same, even though they are very close.